In [150]:
import numpy as np
from sklearn.datasets import fetch_openml
import joblib
###ZA VISE KATEGORIJA
def multiClass():
    
    x, y = fetch_openml('mnist_784', version=1, return_X_y=True, as_frame= False)
    
    y = y.reshape(-1,1).astype(int)
    
    data = np.hstack((y, x))
    
    np.random.seed(42)
    np.random.shuffle(data)
    
    datatrain = data[:50000]
    datadev = data[50000:60000]
    datatest = data[60000:]
    
    print(datatrain.shape, datadev.shape, datatest.shape)
    
    def feature(df):
        return np.hstack((np.ones((df.shape[0],1)), df[:,1:] / 255))
    
    def target(data):
        y = data[:,0].reshape(-1).astype(int)
        y = np.eye(np.max(y) + 1)[y]
        return y
    
    xtrain, xdev, xtest = tuple(feature(skup) for skup in [datatrain, datadev, datatest])
    
    ytrain, ydev, ytest = tuple(target(skup) for skup in [datatrain, datadev, datatest])
    print(xtrain.max())
    print(xtrain.shape, xdev.shape, xtest.shape, ytrain.shape, ydev.shape, ytest.shape)
    return x, y, xtrain, xdev, xtest, ytrain, ydev, ytest

In [151]:
x, y, xtrain, xdev, xtest, ytrain, ydev, ytest = multiClass()

(50000, 785) (10000, 785) (10000, 785)
1.0
(50000, 785) (10000, 785) (10000, 785) (50000, 10) (10000, 10) (10000, 10)


In [152]:
data['train']['x']

array([[1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       ...,
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.]])

In [153]:
xtrain.shape,  ytrain.shape

((50000, 785), (50000, 10))

In [154]:
import sys
import os
sys.path.append(os.path.abspath('../logisticRegression_Numpy'))
from Logit import Logit

In [155]:
Logit?

Init signature: Logit(lr=0.1, regularizacija='', reg=0.01, maxIter=None, nIter=None)
Docstring:      <no docstring>
Init docstring:
Za regularizaciju uneti string 'l1' ili 'l2'
reg je regularizacioni parametar lambda 
learning rate: pocetna brzina ucenja (polovi se na svakih 1000 iteracija) 
File:           c:\users\bogdan.sliskovic\desktop\bole\ml\logisticregression_numpy\logit.py
Type:           type
Subclasses:     

In [156]:
ytrain

array([[0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 1., ..., 0., 0., 0.],
       [0., 0., 0., ..., 1., 0., 0.]])

In [157]:
np.random.seed(42)
w = np.random.rand(xtrain.shape[1],y.shape[1]) - .5

In [158]:
model = Logit()

In [159]:
model.fit(xtrain[:1000],ytrain[:1000], xdev[:10000], ydev[:10000], aktivacija='softmax')

Learning rate: 0.05, iteracija 1000
Optimalni parametri su iz 1500 iteracije
Preciznost na trening setu: 0.978
Preciznost na dev setu: 0.8395


(np.float64(0.978), np.float64(0.8395))

In [292]:
###ZA JEDNU KATEGORIJU
def singleClass():
    x, y = fetch_openml('mnist_784', version=1, return_X_y=True, as_frame= False)
    
    y = y.astype(int).reshape(-1,1)
    y = np.where(y == 5, 1, 0)
    
    y = y.reshape(-1,1)
    
    data = np.hstack((y, x))
    
    np.random.seed(42)
    np.random.shuffle(data)
    
    datatrain = data[:50000]
    datadev = data[50000:60000]
    datatest = data[60000:]
    
    print(datatrain.shape, datadev.shape, datatest.shape)
    
    def feature(df):
        return np.hstack((np.ones((df.shape[0],1)), df[:,1:] / 255))
    
    def target(data):
        y = data[:,0].reshape(-1,1)
        return y
    
    xtrain, xdev, xtest = tuple(feature(skup) for skup in [datatrain, datadev, datatest])
    
    ytrain, ydev, ytest = tuple(target(skup) for skup in [datatrain, datadev, datatest])
    print(xtrain.max())
    print(xtrain.shape, xdev.shape, xtest.shape, ytrain.shape, ydev.shape, ytest.shape)
    
    return x, y, xtrain, xdev, xtest, ytrain, ydev, ytest

In [293]:
x, y, xtrain, xdev, xtest, ytrain, ydev, ytest = singleClass()

(50000, 785) (10000, 785) (10000, 785)
1.0
(50000, 785) (10000, 785) (10000, 785) (50000, 1) (10000, 1) (10000, 1)


In [294]:
y

array([[1],
       [0],
       [0],
       ...,
       [0],
       [1],
       [0]])

In [295]:
model = Logit()

In [296]:
model.fit(xtrain[:1000],ytrain[:1000], xdev[:10000], ydev[:10000], aktivacija='sigmoid')

Learning rate: 0.05, iteracija 1000
Optimalni parametri su iz 1500 iteracije
Preciznost na trening setu: 0.983
Preciznost na dev setu: 0.9553


(np.float64(0.983), np.float64(0.9553))

class NeuralnaMreza(Logit):
    def __init__(self, slojevi, lr=0.1, regularizacija='', reg=0.01, maxIter=10000):
        """slojevi: prima listu"""
        super().__init__(lr=lr, regularizacija=regularizacija, reg=reg, maxIter=maxIter)
        self.slojevi = slojevi
        self.weights = []  # Lista težina za svaki sloj
        for i in range(len(slojevi) - 1):
            W = np.random.randn(slojevi[i], slojevi[i+1]) * 0.01
            self.weights.append(W)
            

In [297]:
x, y, xtrain, xdev, xtest, ytrain, ydev, ytest = multiClass()

(50000, 785) (10000, 785) (10000, 785)
1.0
(50000, 785) (10000, 785) (10000, 785) (50000, 10) (10000, 10) (10000, 10)


## FORWARD PROP

In [301]:
xtrain = xtrain[:,1:]

In [403]:
xtrain = xtrain[:10]
ytrain = ytrain[:10]

In [404]:
xtrain.shape

(10, 784)

In [405]:
logit = Logit()

In [406]:
m,n = xtrain.shape
epsilon = .0000001
np.random.seed(42)

w1 = np.random.rand(xtrain.shape[1], 100) - 0.5
w2 = np.random.rand(100,10) - 0.5

def forward(x, weights, y):
    
    for i, w in enumerate(weights):
        if i == len(weights) - 1:
            x = logit.softmax(x, w)
        else:
            x = logit.sigmoid(x, w)

    loss = -np.sum(y * np.log(x + .000001)) / len(x)
    return loss

In [407]:
L = forward(xtrain, (w1,w2), ytrain)
L

np.float64(3.816428308694275)

## BACK PROP

In [424]:
%%time
epsilon = 1e-4
weights = [w1, w2]
grads = [np.zeros_like(w) for w in weights]

for sloj, w in enumerate(weights):
    g = grads[sloj]

    for i in range(w.shape[0]):
        for j in range(w.shape[1]):
            original = w[i, j]

            w[i, j] = original + epsilon
            fPlus = forward(xtrain, weights, ytrain)


            w[i, j] = original - epsilon
            fMinus = forward(xtrain, weights, ytrain)

            g[i, j] = (fPlus - fMinus) / (2 * epsilon)
            w[i, j] = original


CPU times: total: 1min 46s
Wall time: 2min 37s


In [398]:
weights = [w - 0.01 * g for w, g in zip(weights, grads)]

In [426]:
class NeuralNetwork(Logit):
    def __init__(self, layer_dims, lr=0.1, regularizacija='', reg=0.01, maxIter=None, seed=42):
        super().__init__(lr, regularizacija, reg, maxIter)
        np.random.seed(seed)
        self.weights = [np.random.rand(layer_dims[i], layer_dims[i+1]) - 0.5 for i in range(len(layer_dims)-1)]

    def forward(self, x, y):
        activations = x
        for i, w in enumerate(self.weights):
            if i == len(self.weights) - 1:
                activations = self.softmax(activations, w)
            else:
                activations = self.sigmoid(activations, w)
    
        loss = -np.sum(y * np.log(activations + 1e-6)) / len(x)
    
        if self.regularizacija.lower() == 'l2':
            l2 = (self.reg / (2 * len(x))) * sum([np.sum(w ** 2) for w in self.weights])
            loss += l2
    
        elif self.regularizacija.lower() == 'l1':
            l1 = (self.reg / len(x)) * sum([np.sum(np.abs(w)) for w in self.weights])
            loss += l1
    
        return loss


    def gradijenti(self, x, y, epsilon=1e-4):
        grads = [np.zeros_like(w) for w in self.weights]

        for sloj, w in enumerate(self.weights):
            g = grads[sloj]

            for i in range(w.shape[0]):
                for j in range(w.shape[1]):
                    original = w[i, j]

                    w[i, j] = original + epsilon
                    fPlus = self.forward(x, y)

                    w[i, j] = original - epsilon
                    fMinus = self.forward(x, y)

                    g[i, j] = (fPlus - fMinus) / (2 * epsilon)

                    w[i, j] = original
        return grads
        
    def train_step(self, x, y, epsilon=1e-4, learning_rate=0.01):
        loss = self.forward(x, y)
        grads = self.gradijenti(x, y, epsilon)
        self.weights = [w - learning_rate * g for w, g in zip(self.weights, grads)]
        return loss
